In [1]:
import os

In [2]:
%pwd

'/Users/bhikipallai/Desktop/Projects/Cat_dog/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/bhikipallai/Desktop/Projects/Cat_dog'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class CallbackConfig:
    root_dir: Path
    tensorboard_root_log_dir:Path
    checkpoint_model_filepath:Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list


In [6]:
from src.constants import *
from src.utils.common import read_yaml, create_directories
import tensorflow as tf

In [7]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.data_root])

    
    def get_prepare_callback_config(self) -> CallbackConfig:
        config = self.config.save_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = CallbackConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config
    

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "PetImages")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [8]:
import time

In [9]:
class PrepareCallback:
    def __init__(self, config: CallbackConfig):
        self.config = config

    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )


    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]
    

In [10]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [11]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
    
    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)


    def train(self, callback_list: list):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks=callback_list
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [12]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(
        callback_list=callback_list
    )
    
except Exception as e:
    raise e

[2024-09-05 07:17:56,548: INFO: common]: Yaml file: config/config.yaml loaded successfully
[2024-09-05 07:17:56,550: INFO: common]: Yaml file: params.yaml loaded successfully
[2024-09-05 07:17:56,550: INFO: common]: created directory at: data
[2024-09-05 07:17:56,551: INFO: common]: created directory at: data/prepare_callbacks/checkpoint_dir
[2024-09-05 07:17:56,552: INFO: common]: created directory at: data/prepare_callbacks/tensorboard_log_dir
[2024-09-05 07:17:56,552: INFO: common]: created directory at: data/training
Found 78 images belonging to 2 classes.
Found 318 images belonging to 2 classes.
Epoch 1/25


/Applications/anaconda3/envs/catdog/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


19/19 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - accuracy: 0.5014 - loss: 14.7346 - val_accuracy: 0.3906 - val_loss: 21.5375
Epoch 2/25
 1/19 ━━━━━━━━━━━━━━━━━━━━ 36s 2s/step - accuracy: 0.5625 - loss: 15.6137

2024-09-05 07:18:40.483972: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/Applications/anaconda3/envs/catdog/lib/python3.12/contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


19/19 ━━━━━━━━━━━━━━━━━━━━ 4s 92ms/step - accuracy: 0.5625 - loss: 15.6137 - val_accuracy: 0.0000e+00 - val_loss: 20.2626
Epoch 3/25


...

19/19 ━━━━━━━━━━━━━━━━━━━━ 47s 2s/step - accuracy: 0.6402 - loss: 9.5215 - val_accuracy: 0.3906 - val_loss: 18.8307
Epoch 6/25
19/19 ━━━━━━━━━━━━━━━━━━━━ 4s 94ms/step - accuracy: 0.1875 - loss: 24.3587 - val_accuracy: 0.0000e+00 - val_loss: 66.3088
Epoch 7/25
19/19 ━━━━━━━━━━━━━━━━━━━━ 48s 3s/step - accuracy: 0.5288 - loss: 16.4656 - val_accuracy: 0.3906 - val_loss: 12.9568
Epoch 8/25
19/19 ━━━━━━━━━━━━━━━━━━━━ 4s 98ms/step - accuracy: 0.5000 - loss: 9.5650 - val_accuracy: 0.0000e+00 - val_loss: 28.4368
Epoch 9/25


2024-09-05 07:21:14.389557: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


19/19 ━━━━━━━━━━━━━━━━━━━━ 51s 3s/step - accuracy: 0.5533 - loss: 12.6001 - val_accuracy: 0.6094 - val_loss: 3.4853
Epoch 10/25
19/19 ━━━━━━━━━━━━━━━━━━━━ 4s 99ms/step - accuracy: 0.5000 - loss: 3.0797 - val_accuracy: 0.2143 - val_loss: 12.6651
Epoch 11/25
19/19 ━━━━━━━━━━━━━━━━━━━━ 49s 3s/step - accuracy: 0.5127 - loss: 12.8792 - val_accuracy: 0.3906 - val_loss: 26.0638
Epoch 12/25
19/19 ━━━━━━━━━━━━━━━━━━━━ 4s 108ms/step - accuracy: 0.6875 - loss: 10.3831 - val_accuracy: 1.0000 - val_loss: 3.8912e-06
Epoch 13/25
19/19 ━━━━━━━━━━━━━━━━━━━━ 49s 3s/step - accuracy: 0.6264 - loss: 6.4949 - val_accuracy: 0.6094 - val_loss: 19.6999
Epoch 14/25
19/19 ━━━━━━━━━━━━━━━━━━━━ 4s 102ms/step - accuracy: 0.1875 - loss: 44.5944 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 15/25
19/19 ━━━━━━━━━━━━━━━━━━━━ 49s 3s/step - accuracy: 0.5679 - loss: 11.4175 - val_accuracy: 0.7812 - val_loss: 1.9339
Epoch 16/25
19/19 ━━━━━━━━━━━━━━━━━━━━ 4s 108ms/step - accuracy: 0.7500 - loss: 0.8578 - val_accuracy:

2024-09-05 07:24:49.071683: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


19/19 ━━━━━━━━━━━━━━━━━━━━ 53s 3s/step - accuracy: 0.7427 - loss: 4.2014 - val_accuracy: 0.6562 - val_loss: 8.0627
Epoch 18/25
19/19 ━━━━━━━━━━━━━━━━━━━━ 4s 103ms/step - accuracy: 0.5000 - loss: 10.3630 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 19/25
19/19 ━━━━━━━━━━━━━━━━━━━━ 57s 3s/step - accuracy: 0.5691 - loss: 10.3563 - val_accuracy: 0.8438 - val_loss: 1.8640
Epoch 20/25
19/19 ━━━━━━━━━━━━━━━━━━━━ 4s 116ms/step - accuracy: 0.7500 - loss: 1.7534 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 21/25
19/19 ━━━━━━━━━━━━━━━━━━━━ 53s 3s/step - accuracy: 0.6395 - loss: 7.1264 - val_accuracy: 0.4062 - val_loss: 14.7257
Epoch 22/25
19/19 ━━━━━━━━━━━━━━━━━━━━ 4s 111ms/step - accuracy: 0.5000 - loss: 13.7322 - val_accuracy: 0.2857 - val_loss: 10.3938
Epoch 23/25
19/19 ━━━━━━━━━━━━━━━━━━━━ 51s 3s/step - accuracy: 0.5873 - loss: 8.6328 - val_accuracy: 0.8281 - val_loss: 1.6301
Epoch 24/25
19/19 ━━━━━━━━━━━━━━━━━━━━ 4s 103ms/step - accuracy: 0.7500 - loss: 1.3583 - val_accuracy: 1